# Airline Dataset

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns                       #visualisation
import matplotlib.pyplot as plt             #visualisation
import math
import stats
import warnings
from sklearn.linear_model import LinearRegression
warnings.filterwarnings('ignore')

## 2. Loading the data into the data frame.

In [2]:
df = pd.read_csv("airline_data.csv")
airline_data = df
# To display the top 5 rows
airline_data.head()

,Passanger_Name,Flying_month,Route,Rating,Verified,Review_title,Review_content,Traveller_type,Class
0,Paige Boet,Jun-23,New Orleans to London,1.0,Trip Verified,The airline lost my luggage,The airline lost my luggage and was absolutely...,Solo Leisure,Economy Class
1,S Layne,Mar-23,London to Amman,1.0,Trip Verified,fully refunded by our travel insurance,"We booked on the BA website, round trip flight...",Couple Leisure,Business Class
2,E Lanewoski,Heathrow to Bodrum,Business Class,2.0,Trip Verified,no boarding drinks provided,"First time flying with BA business class, neve...",A321 neo,Solo Leisure
3,Joel Burman,Jun-23,Amman to London,4.0,Not Verified,WiFi didn't work,You can buy sandwiches and crisps but don't ex...,Solo Leisure,Economy Class
4,R Vines,London City to Ibiza,Business Class,7.0,Trip Verified,stick with economy,This is a two-for-one review covering economy ...,Embraer 190,Family Leisure


In [3]:
airline_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3580 entries, 0 to 3579
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Passanger_Name  3580 non-null   object 
 1   Flying_month    2815 non-null   object 
 2   Route           2816 non-null   object 
 3   Rating          3575 non-null   float64
 4   Verified        1270 non-null   object 
 5   Review_title    3580 non-null   object 
 6   Review_content  3580 non-null   object 
 7   Traveller_type  3580 non-null   object 
 8   Class           3579 non-null   object 
dtypes: float64(1), object(8)
memory usage: 251.8+ KB


In [4]:
airline_data.describe().T

,count,mean,std,min,25%,50%,75%,max
Rating,3575.0,4.79049,3.170323,1.0,2.0,4.0,8.0,10.0


# 2. Cleaning Dataset

In [5]:
percentage_null = (airline_data.isnull().sum() / len(airline_data))*100
percentage_null

Passanger_Name     0.000000
Flying_month      21.368715
Route             21.340782
Rating             0.139665
Verified          64.525140
Review_title       0.000000
Review_content     0.000000
Traveller_type     0.000000
Class              0.027933
dtype: float64

In [6]:
airline_data.isnull().sum()

Passanger_Name       0
Flying_month       765
Route              764
Rating               5
Verified          2310
Review_title         0
Review_content       0
Traveller_type       0
Class                1
dtype: int64

In [7]:
# Drop "Verified" column as it has more than 60% of null values 
airline_data.drop('Verified',axis=1, inplace = True)

In [8]:
percentage_null = (airline_data.isnull().sum() / len(airline_data))*100
percentage_null

Passanger_Name     0.000000
Flying_month      21.368715
Route             21.340782
Rating             0.139665
Review_title       0.000000
Review_content     0.000000
Traveller_type     0.000000
Class              0.027933
dtype: float64

In [9]:
airline_data.groupby('Traveller_type').count()                                                         

,Passanger_Name,Flying_month,Route,Rating,Review_title,Review_content,Class
Traveller_type,,,,,,,
767-300,1,1,1,1,1,1,1
777,3,3,3,3,3,3,3
777-200,1,1,1,1,1,1,1
777-300,1,1,1,1,1,1,1
787,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...
Saab 2000,1,1,1,1,1,1,1
Solo Leisure,307,307,307,307,307,307,307
Various,1,1,1,1,1,1,1


### 2.1 Class: Created Class Fixed Feature

In [10]:
airline_data.groupby('Class').count()

,Passanger_Name,Flying_month,Route,Rating,Review_title,Review_content,Traveller_type
Class,,,,,,,
Business,430,430,430,430,430,430,430
Business Class,183,183,183,183,183,183,183
Couple Leisure,641,641,641,641,641,641,641
Economy Class,676,675,676,676,676,676,676
Family Leisure,234,234,234,234,234,234,234
First Class,11,11,11,11,11,11,11
LHR to ORD,1,1,1,1,1,1,1
London to Malaga,1,1,1,1,1,1,1
Los Angeles to London to Paris to Rome,1,1,1,1,1,1,1


In [11]:
#Create New Column to modify as the column could have data usefull for others
airline_data['Class_fix'] = airline_data['Class']

In [12]:
#Copy useful data from Route Column
Route_to_fix = airline_data[(airline_data['Class'] != 'Premium Economy') & (airline_data['Class'] != 'Business Class') & (airline_data['Class'] != 'First Class') & (airline_data['Class'] != 'Economy Class')]['Route']

In [13]:
#Paste Data in the new Class Column, filtering by rows without relevant information for Class
airline_data['Class_fix'][(airline_data['Class_fix'] != 'Premium Economy') & (airline_data['Class_fix'] != 'Business Class') & (airline_data['Class_fix'] != 'First Class_fix') & (airline_data['Class_fix'] != 'Economy Class')] = Route_to_fix

In [14]:
#Copy useful data from Traveller_type
Traveller_to_fix = airline_data[(airline_data['Class_fix'] != 'Premium Economy') & (airline_data['Class_fix'] != 'Business Class') & (airline_data['Class_fix'] != 'First Class') & (airline_data['Class_fix'] != 'Economy Class')]['Traveller_type']

In [15]:
#Paste Data in the new Class Column, filtering by rows without relevant information for Class
airline_data['Class_fix'][(airline_data['Class_fix'] != 'Premium Economy') & (airline_data['Class_fix'] != 'Business Class') & (airline_data['Class_fix'] != 'First Class') & (airline_data['Class_fix'] != 'Economy Class')] = Traveller_to_fix

In [16]:
#Last 13 Rows are first Class, there were a space that produced the error in that Rows, but mnually we can fix it easily
airline_data['Class_fix'][(airline_data['Class_fix'] != 'Premium Economy') & (airline_data['Class_fix'] != 'Business Class') & (airline_data['Class_fix'] != 'First Class') & (airline_data['Class_fix'] != 'Economy Class')] = 'First Class'

In [17]:
#Corrobarting all our values are Class denominatinos
airline_data.groupby('Class_fix').count()

,Passanger_Name,Flying_month,Route,Rating,Review_title,Review_content,Traveller_type,Class
Class_fix,,,,,,,,
Business Class,1161,867,867,1160,1161,1161,1161,1161
Economy Class,1849,1521,1522,1846,1849,1849,1849,1849
First Class,212,156,156,211,212,212,212,211
Premium Economy,358,271,271,358,358,358,358,358


In [18]:
# 2.2 Flying Month

In [19]:
### Flying Month: Created Class Fixed Feature
airline_data.groupby('Flying_month').count()

,Passanger_Name,Route,Rating,Review_title,Review_content,Traveller_type,Class,Class_fix
Flying_month,,,,,,,,
ACC to ZRH via LHR,1,1,1,1,1,1,1,1
AGP to LGW,1,1,1,1,1,1,1,1
AMS to BKK via LHR,1,1,1,1,1,1,1,1
AMS to GRU via LHR,1,1,1,1,1,1,1,1
AMS to HKG via LHR,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...
doha to londonDoha to London,1,1,1,1,1,1,1,1
iAH to LHR,1,1,1,1,1,1,1,1
lgw to alicante,1,1,1,1,1,1,1,1


In [20]:
airline_data['Route'].fillna('None')

0       New Orleans to London
1             London to Amman
2              Business Class
3             Amman to London
4              Business Class
                ...          
3575                     None
3576                     None
3577                     None
3578                     None
3579                     None
Name: Route, Length: 3580, dtype: object

In [21]:
#Create New Column to modify as the column could have data usefull for others
airline_data['Route_fix'] = airline_data['Route']

In [22]:
#Copy useful data from Route Column
Copy_flying = airline_data[(airline_data['Route_fix'] == 'Premium Economy') | (airline_data['Route_fix'] == 'Business Class') | (
                airline_data['Route_fix'] == 'First Class') | (airline_data['Route_fix'] == 'Economy Class')]['Flying_month']

In [23]:
Copy_flying

2                Heathrow to Bodrum
4              London City to Ibiza
6               Amsterdam to London
7       London Heathrow to Kalamata
16               Santiago to London
                   ...             
2808            Gatwick to Barbados
2809                 Kiev to London
2811        London-Vancouver return
2812                        LHR-ORD
2815                            yes
Name: Flying_month, Length: 1858, dtype: object

In [24]:
#Paste Data in the new Class Column, filtering by rows without relevant information for Class
airline_data['Route_fix'][(airline_data['Route_fix'] == 'Premium Economy') | (airline_data['Route_fix'] == 'Business Class') | (
                airline_data['Route_fix'] == 'First Class') | (airline_data['Route_fix'] == 'Economy Class')] = Copy_flying

In [25]:
airline_data

,Passanger_Name,Flying_month,Route,Rating,Review_title,Review_content,Traveller_type,Class,Class_fix,Route_fix
0,Paige Boet,Jun-23,New Orleans to London,1.0,The airline lost my luggage,The airline lost my luggage and was absolutely...,Solo Leisure,Economy Class,Economy Class,New Orleans to London
1,S Layne,Mar-23,London to Amman,1.0,fully refunded by our travel insurance,"We booked on the BA website, round trip flight...",Couple Leisure,Business Class,Business Class,London to Amman
2,E Lanewoski,Heathrow to Bodrum,Business Class,2.0,no boarding drinks provided,"First time flying with BA business class, neve...",A321 neo,Solo Leisure,Business Class,Heathrow to Bodrum
3,Joel Burman,Jun-23,Amman to London,4.0,WiFi didn't work,You can buy sandwiches and crisps but don't ex...,Solo Leisure,Economy Class,Economy Class,Amman to London
4,R Vines,London City to Ibiza,Business Class,7.0,stick with economy,This is a two-for-one review covering economy ...,Embraer 190,Family Leisure,Business Class,London City to Ibiza
...,...,...,...,...,...,...,...,...,...,...
3575,W Benson,NaN,NaN,4.0,British Airways customer review,LHR-HKG on Boeing 747 - 23/08/12. Much has bee...,Economy Class,no,Economy Class,NaN
3576,S Luqman,NaN,NaN,4.0,British Airways customer review,Just got back from Bridgetown Barbados flying ...,Economy Class,no,Economy Class,NaN
3577,D Smith,NaN,NaN,4.0,British Airways customer review,LHR-JFK-LAX-LHR. Check in was ok apart from be...,Economy Class,no,Economy Class,NaN
3578,W Benson,NaN,NaN,6.0,British Airways customer review,HKG-LHR in New Club World on Boeing 777-300 - ...,Business Class,yes,Business Class,NaN


In [43]:
airline_data['Route_fix'][(~airline_data['Route_fix'].str.contains('|'.join(['to','London','LHR','Edinburgh']), na=False)) & (airline_data['Class'] != 'Premium Economy') & (airline_data['Class'] != 'Business Class') & (airline_data['Class'] != 'First Class') & (airline_data['Class'] != 'Economy Class') & (airline_data['Class'] != 'no') & (airline_data['Class'] != 'yes') & (airline_data['Class'] != 'Family Leisure') & (airline_data['Class'] != 'NaN')] = airline_data[(~airline_data['Route_fix'].str.contains('|'.join(['to','London','LHR','Edinburgh']), na=False)) & (airline_data['Class'] != 'Premium Economy') & (airline_data['Class'] != 'Business Class') & (airline_data['Class'] != 'First Class') & (airline_data['Class'] != 'Economy Class') & (airline_data['Class'] != 'no') & (airline_data['Class'] != 'yes') & (airline_data['Class'] != 'Family Leisure') & (airline_data['Class'] != 'NaN')]['Class']

In [50]:
airline_data[(~airline_data['Route_fix'].str.contains('|'.join(['to','London','LHR','Edinburgh']), na=False))]

,Passanger_Name,Flying_month,Route,Rating,Review_title,Review_content,Traveller_type,Class,Class_fix,Route_fix
42,Nicholas Felty,no,May-23,1.0,I had to reschedule my flight,I booked business class tickets for my fiancé ...,Couple Leisure,Business Class,Business Class,May-23
234,Paul Mayer,no,Jun-22,1.0,do not allow free seat selection,"Second time getting poor service from BA. 1st,...",Family Leisure,Business Class,Business Class,Jun-22
419,barbara ciereszko,no,Jul-21,1.0,no refund issued yet,"Booked flight six months before travel, then m...",Family Leisure,Economy Class,Economy Class,Jul-21
2552,Steve Purkis,no,Feb-15,1.0,Fawlty Towers of customer service,My issue is the very poor British Airways cust...,Family Leisure,Economy Class,Economy Class,Feb-15
2567,Travis Rodgers,no,Sep-15,5.0,seats were extremely small,The British Airways seats were extremely small...,Couple Leisure,Economy Class,Economy Class,Sep-15
...,...,...,...,...,...,...,...,...,...,...
3575,W Benson,NaN,NaN,4.0,British Airways customer review,LHR-HKG on Boeing 747 - 23/08/12. Much has bee...,Economy Class,no,Economy Class,NaN
3576,S Luqman,NaN,NaN,4.0,British Airways customer review,Just got back from Bridgetown Barbados flying ...,Economy Class,no,Economy Class,NaN
3577,D Smith,NaN,NaN,4.0,British Airways customer review,LHR-JFK-LAX-LHR. Check in was ok apart from be...,Economy Class,no,Economy Class,NaN
3578,W Benson,NaN,NaN,6.0,British Airways customer review,HKG-LHR in New Club World on Boeing 777-300 - ...,Business Class,yes,Business Class,NaN


In [49]:
route_in_review = ['Heathrow to Tokyo','HER-LGW','Heathrow to San Francisco','Heathrow to Athens','Tampa from Gatwick','Punta Cana - Gatwick','LHR-PEK','EWR to LHR','LGW-PUJ-LGW','Bangkok to London Heathrow','ZRH-LHR','LHR-ATL-LHR','JNB-LHR','LGW-HER','Gatwick to Florida','Gatwick-Naples','Tampa - Gatwick','LGW-CUN-LGW','LGW-RAK-LGW','BLQ-LGW','YYZ-LHR-BRU','LGW-AGP','Madrid-London','LHR to Istanbul','BRU-LHR-YYZ','LAX-LHR-LAX','LHR to CPT','CPT-LHR-MAD','MAN-LHR-YYZ','BRU-LHR-LAX','LGW-MCO-LGW','LHR-EZE','Pisa-LGW','Heathrow to Istanbul','London to Seattle','Manchester to JFK','Gatwick to Barbados','IST-LHR','LCY-ZHR','SAT-DFW-LHR-BRU','LGW-Pisa','LHR-MAD','LHR-SYD-MAD','LHR-SFO-LHR','Heathrow to Bilbao','NCL-LHR','Gatwick to Kingston','Hong Kong to London','LHR-Sidney','LHR-GVA','Sydney-LHR','LHR-BKK','LHR-AMS-LHR','LHR-PHL','LHR to SYD','LHR to Tokyo','Heathrow to Bologna','DEL-LHR','Heathrow to Rome','Delhi to Heathrow','Seattle USA - Heathrow','LGW to LAS','LHR-FRA','MAN-LHR-ATL','MIA-LHR','Bangkok from London','LHR-YVR','LHR Philadelphia','Shanghai to London','Warsaw to Heathrow','LGW-JER-LGW','VIE-LHR','LHR-IAD-LHR','LHR-PHL','DEL-LHR-HAM','GRU-LHR','GVA-LGW-GVA','London to Budapest','LHR-PISA-LHR','LGW-TFS-LGW','Warsaw to Heathrow','LHR-YVR','LGW-JER-LGW','London Heathrow to Dubai','VIE-LHR','LHR-IAD-LHR','LHR-DEL','BRU-EWR','LHR-SIN','LHR-PHL','LHR-LAX','YYZ-DXB','LHR to Frankfurt','LHR to YVR','LHR-Washington','SFO to Milan','IAD-LHR-IAS','Johannesburg to London','Heathrow to Boston','Heathrow to Boston','Heathrow to Washington','LHR-CDG','LAX-LHR','LHR-JFK','Sydney to Singapore','LHR-Zurich','JHB to LHR','LHR-Zurich','LHR-HGK','LHR-MIA','MIA-LHR','MAD-LHR','LGW-ACF','DEN-LHR','NCL-LHR','BKK-LHR','LHR-SIN','DUB-LCY','CPT-JNB','LGW to MCO','SFO-LHR','LGW-MLA','LGW-CUN','LGW-Barcelona','LHR-HKG','Miami to Brussels','LHR to Denver Colorado','MAD-SCL','Manchester to LHR','Los Angeles - Heathrow','LIS-LHR','LGW to TFS','LHR-Prague','HAM-LHR','SAN-MUC','Austin to London','LHR-SFO','London-Prague','TPA - LGW','VIE-LHR','LHR to LAX','Phoenix to London','LHR to IST','LHR-HEL','BGI-LGW','LHR to Seattle','IAH-LHR','YVR-LHR','LHR-BGO','LCA-LHR','London Coty to Florence','NBO-LHR','AMS-LHR','LHR to DXB','MCT to LHR','LHR-ZHR','Kingston to London','LHR-JNB','LGW to Cancun','LGW-ACE','BRU-Toronto','LGW to TFS','LHR to PHX','LHR to Las Vegas','Gatwick to Marrakech','Toronto to London','GIG-LHR','LHR-DEL','IAD-LHR','London to Calgary','OSL-LHR','LGW to Catania','Lucia to London','Aberdeen-Heathrow','LGW to CMB','LHR-MXP','LHR-NCL','LGW to SZG','VIE to LHR','Singapore-London','LHR-FCO','Rome to London Gatwick','LHR-Hannover Germany','LGW to Alicante','Gatwick-Colombia','LHR-NCL','BHD-LHR','LHR-SIS','LHR-SIN','LHR-BUD','Madrid-London','LHR-EWR','LHR-BOS','LHR - IST','LHR-MCT','Rome to LHR','London to Doha','LHR to Muscat','LHR-YYZ','Bari to Gatwick','DXB-LHR','LGW-TFS','LHR-HKK','LHR - Houston','Heathrow to Newark','Porto-LHR','LHR-CPH','LHR-CDG','DUS-LHR','Edinburgh-Gatwick','LHR-BSL','DME-LHR','ZRH to SFO','LAX to ZHR','GOT-LHR','LHR-FRA','LHR to Mumbai','Paris - London','Montreal - LHR','LGW to AMS','LHR-ATH','LHR to BKK','Dublin to Sydney','MCT-LHR','Heathrow - Johannesburg','Gatwick-Venice','']

In [48]:
airline_data[(~airline_data['Route_fix'].str.contains('|'.join(['to','London','LHR','Edinburgh']), na=False)) & (airline_data['Review_content'].str.contains('|'.join(route_in_review), na=False))]

,Passanger_Name,Flying_month,Route,Rating,Review_title,Review_content,Traveller_type,Class,Class_fix,Route_fix
2820,R Vines,NaN,NaN,3.0,British Airways customer review,I flew from Heathrow to Tokyo Haneda in First....,First Class,yes,First Class,NaN
2821,David Ellis,NaN,NaN,2.0,British Airways customer review,HER-LGW. Our return flight confirmed our view ...,Business Class,no,Business Class,NaN
2824,D Evans,NaN,NaN,4.0,British Airways customer review,Flew from Heathrow to San Francisco on May 1st...,Premium Economy,no,Premium Economy,NaN
2825,Sean Kyte,NaN,NaN,7.0,British Airways customer review,Flew Eurotraveller Heathrow to Athens and back...,Economy Class,yes,Economy Class,NaN
3085,Kayleigh Collins,NaN,NaN,3.0,British Airways customer review,Travelled from London Heathrow to Tokyo Haneda...,Economy Class,no,Economy Class,NaN


Siguiente paso, extraer texto de Review content, donde aparecen los destinosy colocarlos en Columna fix

In [ ]:
Extraer de Traveller algunos

In [218]:
#Corrobarting all our values are Class denominatinos
airline_data.groupby('Route_fix').count()

,Passanger_Name,Flying_month,Route,Rating,Review_title,Review_content,Traveller_type,Class,Class_fix
Route_fix,,,,,,,,,
London Heathrow to Madrid,1,1,1,1,1,1,1,1,1
ABV to LHR,1,1,1,1,1,1,1,1,1
ABZ to SFO via LHR,1,1,1,1,1,1,1,1,1
ACC to LHR,1,1,1,1,1,1,1,1,1
AGP to LGW,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...
Zagreb to London,1,1,1,1,1,1,1,1,1
Zagreb to London Heathrow,1,1,1,1,1,1,1,1,1
Zurich to London,5,5,5,5,5,5,5,5,5
